In [12]:
from trulens_eval.app import App
from typing import Any, Callable, ClassVar
from pydantic import Field
from trulens_eval.tru_basic_app import TruWrapperApp
from trulens_eval.util import FunctionOrMethod
from trulens_eval.instruments import Instrument

In [13]:
class FreeFormCallableInstrument(Instrument):

    class Default:
        CLASSES = lambda: {TruWrapperApp}
        METHODS = {"_call": lambda o: isinstance(o, TruWrapperApp)}

    def __init__(self):
        super().__init__(
            root_method=FreeFormApp.call_with_record,
            classes=FreeFormCallableInstrument.Default.CLASSES(),
            methods=FreeFormCallableInstrument.Default.METHODS
        )

In [15]:
class FreeFormApp(App):

    app: TruWrapperApp

    root_callable: ClassVar[FunctionOrMethod] = Field(
        default_factory=lambda: FunctionOrMethod.of_callable(TruBasicApp._call),
        const=True
    )

    def __init__(self, fn: Callable, **kwargs):
        super().update_forward_refs()
        app = TruWrapperApp(fn)
        kwargs['app'] = app
        kwargs['root_class'] = Class.of_object(app)
        kwargs['instrument'] = FreeFormCallableInstrument()
        super().__init__(**kwargs)

    def call_with_record(self, inputs: dict):
        # Wrapped calls will look this up by traversing the call stack. This
        # should work with threads.
        record: Sequence[RecordAppCall] = []

        ret = None
        error = None

        cost: Cost = Cost()

        start_time = None
        end_time = None

        try:
            start_time = datetime.now()
            ret, cost = Endpoint.track_all_costs_tally(
                lambda: self.app._call(input, **kwargs)
            )
            print(record)
            end_time = datetime.now()

        except BaseException as e:
            end_time = datetime.now()
            error = e
            logger.error(f"App raised an exception: {e}")

        assert len(record) > 0, "No information recorded in call."

        ret_record_args = dict()

        ret_record_args['main_input'] = input
        if ret is not None:
            ret_record_args['main_output'] = ret

        ret_record = self._post_record(
            ret_record_args, error, cost, start_time, end_time, record
        )

        return ret, ret_record

In [ ]:
def construct(text: )

In [ ]:
FreeFormApp